<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/colab_usefull_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

functions for using drive with colab

creating different size training data folder

In [0]:
from distutils.dir_util import copy_tree
import os
index = 0
for i,folder in enumerate(os.listdir("/content/drive/My Drive/Datalar100_2")):
  if(folder not in os.listdir("/content/drive/My Drive/data30") and folder != ".DS_Store"):
    if(index<50):
      index += 1
      copy_tree("/content/drive/My Drive/Datalar100_2/"+folder,"/content/drive/My Drive/data50/"+folder)
      print(index)
    else:
      break

dividing training and test sets

In [0]:
import os 
from shutil import copyfile
main_dir = "/content/drive/My Drive/Datalar100_2"

folders = os.listdir(main_dir)
folders.remove(".DS_Store")
test_path = os.path.join(main_dir,"test")
train_path = os.path.join(main_dir,"train")
os.makedirs(test_path)
os.makedirs(train_path)


for i,folder in enumerate(folders):
  try:
    print(i)
    current_test_folder = os.path.join(test_path,folder)
    current_train_folder = os.path.join(train_path,folder)
    os.makedirs(current_test_folder)
    os.makedirs(current_train_folder)
    for index, image in enumerate(os.listdir(os.path.join(main_dir,folder))):
        current_img = os.path.join(main_dir,folder,image)
        if(index<15):
            copyfile(current_img,os.path.join(current_test_folder,image))
        else:
            copyfile(current_img,os.path.join(current_train_folder,image))
  except Exception as e:
    print(e)

**select tf 1.x**

In [0]:
%tensorflow_version 1.x

**keras imports**

In [0]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
import keras

# models
from keras.models import Sequential
from keras.models import Model

# backend
from keras import optimizers, metrics, models
import keras.backend as K

# layers
from keras.layers import Input, add, Conv2D, Flatten, Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

# optimizers
from keras.optimizers import SGD

# regularizers
from keras.regularizers import l2

# training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# keras aplications
from keras.applications import DenseNet201, DenseNet169, InceptionResNetV2, ResNet152V2, InceptionV3, DenseNet121, Xception, MobileNet, VGG19


# other libs
import math
import pickle
import os

**keras image data generator**


In [0]:
from keras.preprocessing.image import ImageDataGenerator

image_size = 224
batch_size = 32


train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.2,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )

valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)


# train = train_datagen.flow_from_directory("/content/drive/My Drive/Data/train", shuffle=True, class_mode='categorical', target_size=(image_size,image_size), batch_size=batch_size)
# valid = valid_datagen.flow_from_directory("/content/drive/My Drive/Data/test", shuffle=True, class_mode='categorical', target_size=(image_size,image_size), batch_size=batch_size)

train = valid_datagen.flow_from_directory("/content/drive/My Drive/clean_data/train", target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory("/content/drive/My Drive/clean_data/test", target_size=(image_size,image_size), batch_size=batch_size)


**convert model to sequential and add last layer**

In [0]:
model = Sequential()
for layer in base_model.layers:
    model.add(layer)

model.add(GlobalAveragePooling2D())
model.add(Dense(100, activation='softmax'))

**optimizer**

In [0]:
opt = SGD(lr=0.001, momentum=0.9)
opt = Adam(learning_rate=0.001)

**model checkpointer and learning schedular**

In [0]:
model_path = "/content/drive/My Drive/model"

checkpointer = ModelCheckpoint(filepath= model_path+'-{epoch:02d}-{val_loss:0.5f}-{val_accuracy:0.5f}.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

def schedule(epoch):
    if epoch < 15:
        return 0.001
    elif epoch < 20:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(schedule)

history = model.fit_generator(train, epochs=100, verbose=1, validation_data=valid, callbacks=[lr_scheduler, checkpointer])